In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
wfrom unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
#     "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
#     "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
#     "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
#     "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
#     "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/gemma-2-9b-bnb-4bit",
#     "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.5: Fast Mistral patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
print("max_seq_length:", max_seq_length, type(max_seq_length))
print("dtype:", dtype, type(dtype))
print("load_in_4bit:", load_in_4bit, type(load_in_4bit))


model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

max_seq_length: 4096 <class 'int'>
dtype: None <class 'NoneType'>
load_in_4bit: True <class 'bool'>


Unsloth 2025.6.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import json

with open("sql_training_dataset.json", "r") as f:
    local_data = json.load(f)


FileNotFoundError: [Errno 2] No such file or directory: 'sql_training_dataset.json'

In [ ]:
from datasets import Dataset

local_dataset = Dataset.from_list(local_data)


In [ ]:
from datasets import concatenate_datasets

In [ ]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

# Load your dataset
d_dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = concatenate_datasets([local_dataset,d_dataset])

# Set up tokenizer with ChatML template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",  # You can change to llama, mistral, etc. if needed
    mapping = {"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
    map_eos_token = True,
)

# Formatting function for ChatML-style messages
def formatting_prompts_func(examples):
    contexts = examples["context"]
    questions = examples["question"]
    answers = examples["answer"]

    conversations = []
    for ctx, q, a in zip(contexts, questions, answers):
        conversations.append([
            {"from": "system", "value": f"Context:\n{ctx.strip()}"},
            {"from": "human", "value": q.strip()},
            {"from": "gpt", "value": a.strip()},
        ])

    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in conversations]
    return {"text": texts}

# Apply the formatting to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)


README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

Unsloth: Will map <|im_end|> to EOS = </s>.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Map:   0%|          | 0/78977 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from transformers import DataCollatorForLanguageModeling

# print("🔍 Tokenizer type before trainer:", type(tokenizer))
# print("🔍 Tokenizer:", tokenizer)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # ❗ THIS IS CRITICAL FOR CAUSAL MODELS
    ),
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/78977 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 78,977 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.393400
2,2.252400
3,2.315600
4,1.914700
5,1.627500
6,1.365100
7,1.545700
8,1.248800
9,1.013400
10,1.072100


In [ ]:
import re

def generate_sql(nl_query, metadata_prompt):
    prompt = f"""### Context ###
{metadata_prompt}

### Task ###
Convert the following natural language query into an SQL query.
Only output the SQL query and a brief explanation, without repeating any of the input.

### User Query ###
{nl_query}

### Output ###
SQL:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)

    # Extract only the SQL query
    sql_pattern = r"### Output ###\s*SQL:\s*(.+?)(?:\n\n|\nExplanation:|\Z)"
    sql_match = re.search(sql_pattern, generated_text, re.DOTALL | re.IGNORECASE)

    if sql_match:
        sql_query = sql_match.group(1).strip()
        if not sql_query.endswith(";"):
            sql_query += ";"
    else:
        sql_query = "Could not parse SQL query."

    return sql_query


# Example usage
nl_query = "I wish to know the name of all employees whose salary is greater than 100 and whose name starts with the letter S"
company_database = "Schema: Employees(id, name, department, salary)"  # Replace with actual schema

result = generate_sql(nl_query, company_database)
print("\nGenerated SQL Query:\n", result)


### Context ###
Schema: Employees(id, name, department, salary)

### Task ###
Convert the following natural language query into an SQL query.
Only output the SQL query and a brief explanation, without repeating any of the input.

### User Query ###
I wish to know the name of all employees whose salary is greater than 100 and whose name starts with the letter S

### Output ###
SQL:
SELECT name FROM Employees WHERE salary > 100 AND name LIKE "S%"

Generated SQL Query:
 SELECT name FROM Employees WHERE salary > 100 AND name LIKE "S%";


In [ ]:
model.save_pretrained("llm_sql_model")  # Local saving
tokenizer.save_pretrained("llm_sql_model")

('llm_sql_model/tokenizer_config.json',
 'llm_sql_model/special_tokens_map.json',
 'llm_sql_model/tokenizer.json')

In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!mv ngrok /usr/local/bin

--2025-04-18 13:18:55--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 13.248.244.96, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9462138 (9.0M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]   9.02M  14.8MB/s    in 0.6s    

2025-04-18 13:18:56 (14.8 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [9462138/9462138]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!ngrok authtoken 2vGcuj6VJaxw6emz5IrnB3mI8l4_3kZa3GS9riotaLguE28N1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install flask_cors

In [ ]:
%%writefile app.py
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
import psycopg2
from flask_cors import CORS
from flask import Flask, request, jsonify

model_path = "llm_sql_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")

def get_db_metadata():
    conn = psycopg2.connect(
        dbname="cdc",
        user="postgres",
        password="password",
        host="0.tcp.in.ngrok.io",
        port="17071"
    )
    cursor = conn.cursor()

    cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
    tables = cursor.fetchall()

    metadata = {}
    for (table,) in tables:
        cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table}';")
        columns = cursor.fetchall()
        metadata[table] = {col[0]: col[1] for col in columns}

    cursor.close()
    conn.close()
    return metadata

def format_metadata(metadata):
    meta_str = "Database Schema:\n"
    for table, columns in metadata.items():
        meta_str += f"Table: {table}\n"
        for col, dtype in columns.items():
            meta_str += f"  - {col} ({dtype})\n"
    return meta_str


def generate_sql(nl_query):
    prompt = f"""### Context ###

   CREATE TABLE Students (
    RNO INT PRIMARY KEY,
    Name TEXT,
    Email TEXT,
    Department TEXT,
    Course TEXT,
    CGPA NUMERIC(4, 2),
    Experience JSONB[],  -- Array of JSON
    Skills TEXT,
    Projects JSONB[],    -- Array of JSON
    Status TEXT[], -- can be 'placed','unplaced' or 'intern'
    Admission_Year INT
);
    CREATE TABLE Companies (
    Company_ID SERIAL PRIMARY KEY,
    Company_Name TEXT,
    Role TEXT,
    Sector TEXT,
    Stipend NUMERIC,
    Website TEXT
);

CREATE TABLE Offers (
    Student_ID INT REFERENCES Students(RNO),
    Company_ID INT REFERENCES Companies(Company_ID),
    Type TEXT, -- can be 'Placement' or 'Internship'
    Status TEXT,  -- can be 'Accepted' , 'Rejected' or 'On Hold'
    Date DATE
);

CREATE TABLE POC (
    Company_ID INT REFERENCES Companies(Company_ID),
    Student_ID INT REFERENCES Students(RNO)
);


    ### Task ###
    Convert the following natural language query into an SQL query. Keep the query as simple and concise as possible so that it still performs its task.
    Only output the SQL query without repeating any of the input.
    Use single quotes (') for all string values in the SQL query, as required by PostgreSQL.

    ### User Query ###
    {nl_query}

    ### Output ###
    SQL:
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.9
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)

    # Improved SQL extraction logic
    sql_pattern = r"### Output ###\s*SQL:\s*(.+?)(?:\n\n|\nExplanation:|\Z)"
    sql_match = re.search(sql_pattern, generated_text, re.DOTALL | re.IGNORECASE)

    if sql_match:
        sql_query = sql_match.group(1).strip()
        if not sql_query.endswith(";"):
            sql_query += ";"
    else:
        sql_query = "Could not parse SQL query."

    return sql_query

# Flask App
app = Flask(__name__)
CORS(app)
metadata = get_db_metadata()
# metadata_prompt = format_metadata(metadata)

@app.route("/generate_sql", methods=["POST"])
def generate_sql_endpoint():
    data = request.json
    user_query = data.get("query", "")

    if not user_query:
        return jsonify({"error": "No query provided"}), 400

    sql_query = generate_sql(user_query)
    return jsonify({"sql_query": sql_query})

@app.route("/execute_query", methods=["POST"])
def execute_query_endpoint():
    data = request.json
    sql_query = data.get("sql_query", "")

    if not sql_query:
        return jsonify({"error": "No SQL query provided"}), 400

    try:
        conn = psycopg2.connect(
            dbname="cdc",
            user="postgres",
            password="password",
            host="0.tcp.in.ngrok.io",
            port="17071"
        )
        cursor = conn.cursor()

        cursor.execute(sql_query)

        result = None
        # Check if it's a SELECT query to fetch results
        if sql_query.strip().lower().startswith("select"):
            result = cursor.fetchall()
        else:
            return jsonify({"result": "Not a SELECT query"})
        print(result)
        conn.commit()
        return jsonify({"result": result if result is not None else "Query executed successfully"})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

if __name__ == "__main__":
    app.run(port=5000, host="0.0.0.0")


Overwriting app.py


In [ ]:
!nohup python3 app.py > output.log 2>&1 &

In [ ]:
!nohup ngrok http 5000 > ngrok.log 2>&1 &

In [ ]:
import time
import requests

time.sleep(3)

try:
    response = requests.get("http://localhost:4040/api/tunnels")
    tunnels = response.json()["tunnels"]
    public_url = tunnels[0]["public_url"]
    print("Ngrok Public URL:", public_url)
except Exception as e:
    print("Error getting ngrok URL:", e)


Ngrok Public URL: https://94cf-34-82-249-73.ngrok-free.app


In [ ]:
import requests

url = "https://94cf-34-82-249-73.ngrok-free.app/generate_sql"

payload = {
    "query": "Total number of Computer Science and Engineering students who got placed."
}

response = requests.post(url, json=payload)

if response.status_code == 200:
    print("Generated SQL Query:", response.json().get("sql_query"))
else:
    print("Error:", response.json())

query = response.json().get("sql_query")

query = query.replace('"', "'")
print("QUery is :",query)
url_exec = "https://94cf-34-82-249-73.ngrok-free.app/execute_query"

payload = {
    "sql_query": query
}

response = requests.post(url_exec, json=payload)

if response.status_code == 200:
    print("Response:", response.json().get("result"))
else:
    print("Error:", response.json())



Generated SQL Query: SELECT COUNT(*) FROM Students
     WHERE Department = "Computer Science and Engineering"
     AND Status = "placed";
QUery is : SELECT COUNT(*) FROM Students
     WHERE Department = 'Computer Science and Engineering'
     AND Status = 'placed';
Response: [[34]]


In [ ]:
!ps aux | grep app.py

root        3651  4.1 14.8 49871936 1969888 ?    Sl   13:10   0:20 python3 app.py
root        5655  0.0  0.0   7376  3596 ?        S    13:18   0:00 /bin/bash -c ps aux | grep app.py
root        5657  0.0  0.0   6484  2356 ?        S    13:18   0:00 grep app.py


In [ ]:
!kill -9 3651

In [ ]:
!ps aux | grep ngrok

root        3927  0.2  0.2 1254824 30756 ?       Sl   13:11   0:00 ngrok http 5000
root        5699  0.0  0.0   7376  3428 ?        S    13:18   0:00 /bin/bash -c ps aux | grep ngrok
root        5701  0.0  0.0   7376   288 ?        R    13:18   0:00 /bin/bash -c ps aux | grep ngrok


In [ ]:
!kill -9 3927

In [ ]:
import requests

url = "https://94cf-34-82-249-73.ngrok-free.app/generate_sql"

payload = {
    "query": "Name of students who got offers from Baker Inc"
}

response = requests.post(url, json=payload)

if response.status_code == 200:
    print("Generated SQL Query:", response.json().get("sql_query"))
else:
    print("Error:", response.json())

query = response.json().get("sql_query")
query = query.replace('"', "'")
print("QUery is :",query)

url_exec = "https://94cf-34-82-249-73.ngrok-free.app/execute_query"

payload = {
    "sql_query": query
}

response = requests.post(url_exec, json=payload)

if response.status_code == 200:
    print("Response:", response.json().get("result"))
else:
    print("Error:", response.json())

Generated SQL Query: SELECT Name FROM Students JOIN Offers ON Students.RNO = Offers.Student_ID JOIN Companies ON Offers.Company_ID = Companies.Company_ID WHERE Companies.Company_Name = "Baker Inc";
QUery is : SELECT Name FROM Students JOIN Offers ON Students.RNO = Offers.Student_ID JOIN Companies ON Offers.Company_ID = Companies.Company_ID WHERE Companies.Company_Name = 'Baker Inc';
Response: [['Dawn Chapman'], ['Danielle West'], ['Barbara Peterson'], ['Eileen Clayton'], ['Carl Crawford'], ['Harry Villegas']]


In [ ]:
import requests

url = "https://94cf-34-82-249-73.ngrok-free.app/generate_sql"

payload = {
    "query": "List of all offers offered to students under the POC 'Carmen Taylor'?"
}

response = requests.post(url, json=payload)

if response.status_code == 200:
    print("Generated SQL Query:", response.json().get("sql_query"))
else:
    print("Error:", response.json())

query = response.json().get("sql_query")
query = query.replace('"', "'")
print("QUery is :",query)

url_exec = "https://94cf-34-82-249-73.ngrok-free.app/execute_query"

payload = {
    "sql_query": query
}

response = requests.post(url_exec, json=payload)

if response.status_code == 200:
    print("Response:", response.json().get("result"))
else:
    print("Error:", response.json())

Generated SQL Query: SELECT * FROM Offers
     JOIN Students ON Offers.Student_ID = Students.RNO
     JOIN Companies ON Offers.Company_ID = Companies.Company_ID
     JOIN POC ON Companies.Company_ID = POC.Company_ID
     WHERE POC.Student_ID = Students.RNO AND POC.Student_ID = 1000000000 AND POC.Company_ID = 1000000001;
QUery is : SELECT * FROM Offers
     JOIN Students ON Offers.Student_ID = Students.RNO
     JOIN Companies ON Offers.Company_ID = Companies.Company_ID
     JOIN POC ON Companies.Company_ID = POC.Company_ID
     WHERE POC.Student_ID = Students.RNO AND POC.Student_ID = 1000000000 AND POC.Company_ID = 1000000001;
Response: []
